In [3]:
import numpy as np

# 자연어 처리
- 우리의 말을 컴퓨터에게 이해시키기 위한 기술
- 검색 엔진, 기계 번역, 질의응답 시스템, IME, 문장 자동요약, 감정 분석

## 시소러스
- 단어의 의미를 사람이 정의하는 기법
- thesaurus (유의어 사전) 을 활용하는 방법
- 뜻이 비슷한 단어가 한 그룹으로 분류되어 있음

## 통계 기반 기법
- 말뭉치
- 자연어 처리 연구나 애플리케이션을 염두에 두고 수집된 텍스트 데이터

In [1]:
text = 'you say goodbye and I say hello.'

text = text.lower()

word = text.split(' ')

In [2]:
word

['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello.']

In [3]:
word_to_idx = {}
idx_to_word = {}

for idx, w in enumerate(word) :
    word_to_idx[w] = idx
    idx_to_word[idx] = w

In [4]:
word_to_idx, idx_to_word

({'you': 0, 'say': 5, 'goodbye': 2, 'and': 3, 'i': 4, 'hello.': 6},
 {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'say', 6: 'hello.'})

In [5]:
def preprocess(text) :
    text = text.lower()
    #regx 로 문장 부호 분리 필요
    text = text.replace('.', ' .')
    words = text.split(' ')
    
    word_to_idx = {}
    idx_to_word = {}

    for idx, w in enumerate(word) :
        word_to_idx[w] = idx
        idx_to_word[idx] = w
        
    corpus = np.array([word_to_idx[w] for i in words])
    
    return corpus, word_to_idx, idx_to_word

## 단어의 분산 표현

- 단어의 의미를 정확하게 표현할 수 있는 벡터 표현
- co-occurence matrix 를 만든다. (-> 주변 맥락(window) 를 통해 확인)
- 분포 가설

#### distributional hypothesis (분포 가설)
- words that are used and occur in the same contexts tend to purport similar meanings
- a word is characterized by the company it keeps (1950s)

In [4]:
C = np.array([
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0]
], dtype=np.int32)

In [5]:
#해당 단어가 다른 단어의 window에 속한 횟수
np.sum(C, axis=0)

array([1, 4, 2, 2, 2, 2, 1])

## 벡터 간 유사도
- 코사인 유사도
    - 분모는 L2 norm 제곱합의 제곱근(root)
    - 분자는 벡터 간의 내적합
- eps : 엡실론
    - for division by zero error

In [ ]:
def cos_similarity(x, y, eps = 1e-8) :
    nx = x / np.sqrt(np.sum(x**2)+eps)
    ny = y / np.sqrt(np.sum(y**2)+eps)
    
    return np.dot(nx, xy)

### most similar 함수

In [1]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5) :
    
    if query not in word_to_id :
        print("%s not found" %query)
        return

    print('\n[query] ' + query)
    
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    
    for i in range(vocab_size) :
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
        
    count = 0
    
    for i in (-1 * similarity).argsort() :
        if id_to_word[i] == query :
            continue
        
        print(" %s : %s" %(id_to_word[i], similarity[i]))
        
        count += 1
        
        if count >= top :
            return

## 통계 기반 기법 개선하기
#### 상호정보량
  - 단순 동시 발생 빈도수로 벡터를 만들면 고빈도 단어(관사와 같은) 는 모든 단어와 관련성이 높게 나타난다.
  - 따라서 PMI 라는 지수를 활용해 벡터를 만든다.
  - $$ PMI(x, y) = log_2 \cfrac{C(x, y)*N}{C(x)C(y)} $$
  - 양의 상호정보량 $$ PPMI(x, y) = max(0, PMI(x, y)) $$

In [6]:
def ppmi(C, verbose = False, eps = 1e-8) :
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis = 0)
    
    total = C.shape[0] * C.shape[1]
    cnt = 0
    
    for i in range(C.shape[0]) :
        for j in range(C.shape[1]) :
            pmi = np.log2(C[i, j] * N / (S[i]*S[j])+eps)
            M[i, j] = max(0, pmi)
            
            if verbose :
                cnt += 1
                if cnt % (total//100) == 0 :
                    print('%.1f%% 완료' % (100*cnt/total))
    return M

- 문제 존재. : 단어마다 고유 인덱스. 단어의 수가 많을수록 행렬의 크기도 커진다

### 특이값 분해 (singular value decomposition)
- 차원 축소를 위한 방법
- u 직교 행렬(단어 공간의 기저) / s 특이값을 대각성분으로 한 대각행렬
- u 와 v는 직교함

In [7]:
u, s, v = np.linalg.svd(C)

In [8]:
u, s, v

(array([[ 0.00000000e+00, -3.63987887e-01,  7.77156117e-16,
         -1.97083658e-01,  2.22044605e-16, -9.10313600e-01,
          4.86171902e-17],
        [ 8.46041188e-01,  0.00000000e+00,  2.26091196e-01,
          0.00000000e+00, -4.82801284e-01,  0.00000000e+00,
          0.00000000e+00],
        [-4.48640785e-17, -5.77929845e-01,  8.30926981e-16,
          3.79210349e-01,  3.10497138e-16,  1.48985251e-01,
         -7.07106781e-01],
        [ 4.97279485e-01, -7.65004295e-16, -6.61115197e-01,
         -1.29942762e-16,  5.61818307e-01, -7.46273025e-17,
          1.54500638e-17],
        [-4.54404361e-17, -5.77929845e-01,  8.41601691e-16,
          3.79210349e-01,  3.14486016e-16,  1.48985251e-01,
          7.07106781e-01],
        [-4.54404361e-17, -4.46662072e-01,  8.41601691e-16,
         -8.20705217e-01,  3.14486016e-16,  3.56280704e-01,
          2.14358315e-18],
        [ 1.92165093e-01,  1.47712142e-15,  7.15408601e-01,
          2.12584375e-16,  6.71761200e-01,  2.40242105e-16

#### 차원 축소
- s 특이값이 큰 값만 살려도 되지 않을까? n번째 큰값으로만 해당 행렬을 복원해낼 수 있음

In [9]:
s.argsort()

array([6, 5, 4, 3, 2, 0, 1], dtype=int64)

In [10]:
W = ppmi(C)

#SVD
U, S, V = np.linalg.svd(W)